a parte de limpesa de dados ela e relacionada e variada ou seja ela varia bastante

libs

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score, roc_curve, auc
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.feature_selection import SelectKBest, f_classif
from dataclasses import dataclass
from google.colab import files

classe de leitura do dataset, estamos usando a entrada de arquivo do google colab mas pode ser editado para um caminho do seu sistema

In [12]:
def read_my_dataset(csv_path: str) -> pd.DataFrame:
    """
    Lê um arquivo CSV contendo colunas:
    partida_id, rodata, clube, cartao, atleta, num_camisa, posicao, minuto.

    Retorna um DataFrame do pandas com essas colunas.
    """
    data = pd.read_csv(csv_path)
    print("Colunas encontradas no dataset:", data.columns)

    # Exemplo de acesso às colunas
    print("\nPrimeiras linhas do dataset:")
    print(data.head())

    return data

desing patterns - factory com a implementação de 7 algoritimos, podendo gerar facil manutenção para adicionar ou remover

In [13]:
def get_models():
    """
    Retorna uma lista de modelos com configurações padrão.
    """
    models = []
    models.append(Model("Regressão Logística",
                        LogisticRegression(max_iter=1000, random_state=42)))
    models.append(Model("Árvore de Decisão",
                        DecisionTreeClassifier(max_depth=5, random_state=42)))
    models.append(Model("KNN",
                        KNeighborsClassifier(n_neighbors=5)))
    models.append(Model("Naive Bayes (Gaussian)",
                        GaussianNB()))
    models.append(Model("SVM",
                        SVC(probability=True, random_state=42)))
    models.append(Model("Rede Neural (MLP)",
                        MLPClassifier(hidden_layer_sizes=(100, 50),
                                      max_iter=1000,
                                      random_state=42)))
    models.append(Model("Random Forest",
                        RandomForestClassifier(n_estimators=100, random_state=42)))
    models.append(Model("Gradient Boosting",
                        GradientBoostingClassifier(n_estimators=100, random_state=42)))
    if xgboost_available:
        models.append(Model("XGBoost",
                            XGBClassifier(n_estimators=100, random_state=42)))
    return models

plot de matriz de confusao

In [14]:

def plot_confusion_matrix(y_true, y_pred, title="Matriz de Confusão"):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(4, 3))
    sns.heatmap(cm, annot=True, cmap="Blues", fmt="d")
    plt.title(title)
    plt.xlabel("Previsto")
    plt.ylabel("Verdadeiro")
    plt.show()


metricas de avaliação ROC

conhecido como Curva de característica de operação do receptor

### 1. True Positive Rate (TPR / Sensibilidade)
A **taxa de verdadeiros positivos** indica o quão bem o modelo detecta os positivos reais.
Fórmula:
$$
\text{TPR} = \frac{TP}{TP + FN}
$$
Onde:  
- **TP** = True Positives (Verdadeiros Positivos)  
- **FN** = False Negatives (Falsos Negativos)
---
### 2. False Positive Rate (FPR)
A **taxa de falsos positivos** mostra com que frequência o modelo classifica incorretamente os negativos como positivos.
Fórmula:
$$
\text{FPR} = \frac{FP}{FP + TN}
$$
Onde:  
- **FP** = False Positives (Falsos Positivos)  
- **TN** = True Negatives (Verdadeiros Negativos)

**Relação com a Área Sob a Curva (AUC-ROC)**

A **AUC-ROC** (Área Sob a Curva ROC) mede a capacidade do modelo de distinguir entre classes. Quanto maior a AUC (próxima de 1), melhor o modelo. A fórmula da AUC é calculada integrando a curva ROC:

$$
\text{AUC} = \int_{0}^{1} \text{TPR}(\text{FPR}) \, d(\text{FPR})
$$

In [15]:
#analisar a area do grafico do plot
def plot_roc_curve(y_true, y_scores, title="Curva ROC"):
    fpr, tpr, _ = roc_curve(y_true, y_scores)
    roc_auc = auc(fpr, tpr)
    plt.figure(figsize=(5, 4))
    plt.plot(fpr, tpr, color="darkorange", lw=2, label=f"AUC = {roc_auc:.2f}")
    plt.plot([0, 1], [0, 1], color="navy", lw=2, linestyle="--")
    plt.xlabel("Falso Positivo")
    plt.ylabel("Verdadeiro Positivo")
    plt.title(title)
    plt.legend(loc="lower right")
    plt.show()

preparação do piperline

filtrando a parte de valores com NULL e NAN e removendo duplicatas

In [16]:
#talvez seja melhor usar uma lib para a analise exploratoria de dados

def pipeline_cartoes():
    """
    Pipeline para análise e predição de cartões em partidas de futebol
    usando o dataset com colunas: partida_id, rodata, clube, cartao, atleta,
    num_camisa, posicao, minuto.
    """
    # ----------------------------------------------------------------
    # Passo 1: Carregar os dados
    # ----------------------------------------------------------------
    try:
        uploaded = files.upload()
        dataset_path = list(uploaded.keys())[0]
        print(f"Arquivo carregado: {dataset_path}")
    except:
        print("Erro ao carregar arquivo. Usando caminho fixo.")
        dataset_path = "seu_dataset.csv"  # Substitua pelo caminho correto

    data = read_my_dataset(dataset_path)

    # ----------------------------------------------------------------
    # Passo 2: Limpeza e Exploração de Dados
    # ----------------------------------------------------------------
    print("\n=== Explorando dados de cartões ===")

    # Verificar valores nulos
    print("\nValores nulos por coluna:")
    print(data.isnull().sum())

    # Remover duplicatas
    data_cleaned = data.drop_duplicates()
    print(f"\nLinhas após remover duplicatas: {data_cleaned.shape[0]} (original: {data.shape[0]})")

    # Preencher valores nulos se necessário
    for col in data_cleaned.columns:
        if data_cleaned[col].isnull().sum() > 0:
            if data_cleaned[col].dtype in ['float64', 'int64']:
                data_cleaned[col].fillna(data_cleaned[col].mean(), inplace=True)
            else:
                data_cleaned[col].fillna(data_cleaned[col].mode()[0], inplace=True)

    # Análise básica dos cartões
    if 'cartao' in data_cleaned.columns:
        print("\nDistribuição de cartões:")
        cartoes_count = data_cleaned['cartao'].value_counts()
        print(cartoes_count)

        plt.figure(figsize=(8, 4))
        sns.countplot(x='cartao', data=data_cleaned)
        plt.title('Distribuição de Cartões')
        plt.xlabel('Tipo de Cartão')
        plt.ylabel('Quantidade')
        plt.show()

    # Análise por posição (se disponível)
    if 'posicao' in data_cleaned.columns:
        print("\nCartões por posição:")
        cartoes_posicao = pd.crosstab(data_cleaned['posicao'], data_cleaned['cartao'])
        print(cartoes_posicao)

        plt.figure(figsize=(10, 6))
        cartoes_posicao.plot(kind='bar', stacked=True)
        plt.title('Cartões por Posição')
        plt.xlabel('Posição')
        plt.ylabel('Quantidade')
        plt.legend(title='Tipo de Cartão')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()
         # Codificar variáveis categóricas
    data_encoded = data_cleaned.copy()
    categorical_columns = data_cleaned.select_dtypes(include=['object']).columns

    label_encoders = {}
    for col in categorical_columns:
        le = LabelEncoder()
        data_encoded[col] = le.fit_transform(data_cleaned[col].astype(str))
        label_encoders[col] = le
        print(f"Codificação para {col}: {dict(zip(le.classes_, le.transform(le.classes_)))}")

    # Definir variável alvo (neste caso vamos prever o tipo de cartão)
    if 'cartao' in data_encoded.columns:
        target = 'cartao'
        print(f"\nPrevisão da variável alvo: {target}")
        X = data_encoded.drop(columns=[target])
        y = data_encoded[target]

        # Dividir em treino e teste
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.3, stratify=y, random_state=42
        )

        print(f"Tamanho do conjunto de treino: {X_train.shape[0]} amostras")
        print(f"Tamanho do conjunto de teste: {X_test.shape[0]} amostras")

        # Normalizar as features numéricas
        numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns
        if len(numeric_cols) > 0:
            scaler = MinMaxScaler()
            X_train[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
            X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])

preparacao das melhores features com o kbeast

In [17]:
selector = SelectKBest(score_func=f_classif, k='all')
        selector.fit(X_train, y_train)

        # Obter scores de importância
        feature_scores = pd.DataFrame({
            'Feature': X.columns,
            'Score': selector.scores_
        })
        feature_scores = feature_scores.sort_values('Score', ascending=False)
        print("\nImportância das features:")
        print(feature_scores.head(10))  # Top 10 features

        plt.figure(figsize=(10, 6))
        sns.barplot(x='Score', y='Feature', data=feature_scores.head(10))
        plt.title('Top 10 Features Mais Importantes')
        plt.tight_layout()
        plt.show()

        # Selecionar as top features (opcional)
        k_best = min(10, len(X.columns))  # Limitar a 10 ou menos
        top_features = feature_scores['Feature'].head(k_best).values
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]

        print(f"\nUsando as top {k_best} features para modelagem")


IndentationError: unexpected indent (<ipython-input-17-5c5f4c6c3280>, line 2)

treinamento e avaliacao dos modelos

In [ ]:
 print("\n=== Treinamento e avaliação de modelos ===")
        models = get_models()
        results = []

        for model in models:
            print(f"\nTreinando {model.nome}...")
            clf = model.estimator
            clf.fit(X_train_selected, y_train)
            y_pred = clf.predict(X_test_selected)

        #usando o precision e recall pelo metivo do banco de dado
            acc = accuracy_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred, average='weighted')
            precision = precision_score(y_test, y_pred, average='weighted')
            recall = recall_score(y_test, y_pred, average='weighted')

            results.append({
                'Modelo': model.nome,
                'Acurácia': acc,
                'F1 Score': f1,
                'Precisão': precision,
                'Recall': recall
            })

            print(f"Acurácia: {acc:.4f}")
            print(f"F1 Score: {f1:.4f}")

            # Matriz de confusão
            plot_confusion_matrix(y_test, y_pred, title=f"Matriz de Confusão - {model.nome}")

            # Importância das features (para modelos que suportam)
            if hasattr(clf, 'feature_importances_'):
                importances = pd.DataFrame({
                    'Feature': top_features,
                    'Importance': clf.feature_importances_
                }).sort_values('Importance', ascending=False)

                plt.figure(figsize=(10, 6))
                sns.barplot(x='Importance', y='Feature', data=importances)
                plt.title(f'Importância das Features - {model.nome}')
                plt.tight_layout()
                plt.show()

comparacao dos modelos

In [ ]:
 results_df = pd.DataFrame(results)
        print("\n=== Comparação dos Modelos ===")
        print(results_df)

        plt.figure(figsize=(12, 6))
        results_df_melted = pd.melt(results_df,
                                   id_vars=['Modelo'],
                                   value_vars=['Acurácia', 'F1 Score', 'Precisão', 'Recall'],
                                   var_name='Métrica', value_name='Valor')
        sns.barplot(x='Modelo', y='Valor', hue='Métrica', data=results_df_melted)
        plt.title('Comparação de Modelos')
        plt.xticks(rotation=45)
        plt.ylim(0, 1)
        plt.tight_layout()
        plt.show()
    else:
        print("A coluna 'cartao' não foi encontrada no dataset.")

plots dos algoritimos

In [ ]:
   # Plot da Curva ROC (para binário)
        if hasattr(clf, "predict_proba"):
            y_prob = clf.predict_proba(X_test)[:, 1]
            plot_roc_curve(y_test, y_prob, title=f"{model.nome} - Curva ROC")

        # Exemplo de visualização da árvore
        if isinstance(clf, DecisionTreeClassifier):
            plt.figure(figsize=(10, 6))
            plot_tree(clf, filled=True, feature_names=[f"feat_{i}" for i in range(X.shape[1])])
            plt.title(f"{model.nome} - Árvore de Decisão")
            plt.show()

        # Importância de features (RandomForest, GradientBoosting, XGBoost)
        if isinstance(clf, (RandomForestClassifier, GradientBoostingClassifier)):
            importances = clf.feature_importances_
            indices = np.argsort(importances)[::-1]
            plt.figure(figsize=(6, 4))
            plt.bar(range(X.shape[1]), importances[indices], color="r", align="center")
            plt.xticks(range(X.shape[1]), [f"feat_{i}" for i in indices], rotation=45)
            plt.title(f"{model.nome} - Importância das Features")
            plt.tight_layout()
            plt.show()

        if xgboost_available and isinstance(clf, XGBClassifier):
            importances = clf.feature_importances_
            indices = np.argsort(importances)[::-1]
            plt.figure(figsize=(6, 4))
            plt.bar(range(X.shape[1]), importances[indices], color="g", align="center")
            plt.xticks(range(X.shape[1]), [f"feat_{i}" for i in indices], rotation=45)
            plt.title(f"{model.nome} - Importância das Features (XGBoost)")
            plt.tight_layout()
            plt.show()

    print(f"\nMelhor modelo no dataset aleatório: {melhor_modelo} com acurácia de {melhor_acc:.4f}")

In [ ]:

if __name__ == "__main__":
    print("=== Pipeline para Análise de Cartões em Partidas de Futebol ===")
    print("Carregue seu dataset com colunas: partida_id, rodata, clube, cartao, atleta, num_camisa, posicao, minuto")
    pipeline_cartoes()
